In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

2023-07-14 08:32:48.172391: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Loading data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [5]:
# Preprocessing data
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)) / 255.0
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)) / 255.0

In [6]:
# Apply data augmentation
datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
datagen.fit(x_train)

In [7]:
# Learning rate scheduler
def lr_schedule(epoch):
    initial_learning_rate = 0.001
    decay_rate = 0.9
    decay_steps = 5
    lr = initial_learning_rate * decay_rate**(epoch // decay_steps)
    return lr

In [8]:
# Build the model within the strategy's scope
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

2023-07-14 08:33:46.868857: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 


In [9]:
# Compile the model
model.compile(optimizer=Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# Create a learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

In [11]:
# Train model
model.fit(datagen.flow(x_train, y_train, batch_size=64), epochs=10, validation_data=(x_test, y_test))

2023-07-14 08:33:47.429350: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/10
938/938 [==============================] - 310s 329ms/step - loss: 0.6660 - accuracy: 0.7516 - val_loss: 0.4045 - val_accuracy: 0.8523
Epoch 2/10
938/938 [==============================] - 328s 350ms/step - loss: 0.4591 - accuracy: 0.8328 - val_loss: 0.3546 - val_accuracy: 0.8662
Epoch 3/10
938/938 [==============================] - 322s 343ms/step - loss: 0.3887 - accuracy: 0.8575 - val_loss: 0.3085 - val_accuracy: 0.8896
Epoch 4/10
938/938 [==============================] - 329s 350ms/step - loss: 0.3591 - accuracy: 0.8695 - val_loss: 0.2973 - val_accuracy: 0.8962
Epoch 5/10
938/938 [==============================] - 313s 334ms/step - loss: 0.3380 - accuracy: 0.8759 - val_loss: 0.3419 - val_accuracy: 0.8806
Epoch 6/10
938/938 [==============================] - 314s 335ms/step - loss: 0.3228 - accuracy: 0.8811 - val_loss: 0.2704 - val_accuracy: 0.9026
Epoch 7/10
938/938 [==============================] - 309s 329ms/step - loss: 0.3077 - accuracy: 0.8881 - val_loss: 0.2775 -

In [12]:
# Evaluate model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 2s 8ms/step - loss: 0.2547 - accuracy: 0.9101
Test accuracy: 0.910099983215332


In [ ]:
# Save the model
model.save('final_model.h5')